In [1]:
!pip install -q datasets transformers bitsandbytes accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
# AutoModelForCausalLM - to get model
# AutoTokenizer - to convert to tokens
# TrainingArguments - model train panna thevayana parameters kuduthukalam
# Trainer - Before fine-tuning, adhuve train panna

from peft import get_peft_model, LoraConfig, TaskType
# LoraConfig - lora model parameters kaga
# TaskType - what type of task we do

from datasets import Dataset

In [3]:
data = {
    "text": {
        "### Question: Who is Virat Kohli?\n### Answer: Virat Kohli is a great match winner",
        "### Question: Who is MS Dhoni?\n### Answer: MS Dhoni is a man who owns victory like no one else",
        "### Question: Who is Sachin Tendulkar?\n### Answer: Sachin is the God of Cricket."
    }
}
dataset = Dataset.from_dict(data)

In [4]:
# Falcon - LLM model
model_name = "tiiuae/falcon-rw-1b"

# Integrating Autotokenizer with model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token     # tokenizer la dhan model and tokenizer rendume iruku

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [5]:
def tokenize(batch):
  tokens = tokenizer(
      batch["text"],
      padding = "max_length",
      truncation = True,      # Chunks ah pirika, 128 mela works pona, chunks ah pirikum
      max_length = 128
  )
  tokens["labels"] = tokens["input_ids"].copy()    # Labels - endha word munnadi iruku, bc while training words swap aaga kudadhu
  return tokens

tokenized_dataset = dataset.map(tokenize, batched = True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit = True,
    device_map = "auto"   # Related to hardware for switching b/w CPU & GPU
)

config.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [7]:
# Initializing LoRA
lora_config = LoraConfig(
    r = 8,       # r - adapters
    lora_alpha = 16,   # Scaling
    target_modules = ["query_key_value"],  # [LoRA Working] query_key_value - Attention Mechanism la 1 la iruka values mattum fine tune panna
    lora_dropout = 0.1,     # To reduce nodes since nodes and complexity may keep increasing
    bias = "none",
    task_type = TaskType.CAUSAL_LM
)

In [8]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [9]:
training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 3,
    per_device_train_batch_size = 1,
    learning_rate = 2e-4,
    logging_steps = 1,
    save_strategy = "no",
    report_to = "none"   # Notify panna
)

In [10]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    tokenizer = tokenizer
)

trainer.train()

/tmp/ipython-input-10-1711084281.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,5.456200
2,5.111800
3,4.648700
4,4.207500
5,3.417300
6,3.070700
7,2.613300
8,2.379600
9,2.159000


TrainOutput(global_step=9, training_loss=3.673789527681139, metrics={'train_runtime': 7.0052, 'train_samples_per_second': 1.285, 'train_steps_per_second': 1.285, 'total_flos': 8364732973056.0, 'train_loss': 3.673789527681139, 'epoch': 3.0})

In [11]:
model.save_pretrained("lora-finetuned")      # Direct ah save panradhu
tokenizer.save_pretrained("lora-finetuned")  # Tokenize panni save panradhu

('lora-finetuned/tokenizer_config.json',
 'lora-finetuned/special_tokens_map.json',
 'lora-finetuned/vocab.json',
 'lora-finetuned/merges.txt',
 'lora-finetuned/added_tokens.json',
 'lora-finetuned/tokenizer.json')